In [ ]:
from langchain_core.example_selectors import LengthBasedExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
example_selector = LengthBasedExampleSelector(
    # The examples it has available to choose from.
    examples=examples,
    # The PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # The maximum length that the formatted examples should be.
    # Length is measured by the get_text_length function below.
    max_length=25,
    # The function used to get the length of a string, which is used
    # to determine which examples to include. It is commented out because
    # it is provided as a default value if none is specified.
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)

dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="获取输入内容的反义词",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

def test1():
    # An example with small input, so it selects all examples.
    print(dynamic_prompt.format(adjective="big"))


def test2():
   # An example with long input, so it selects only one example.
    long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
    print(dynamic_prompt.format(adjective=long_string))


def test3():
    # You can add an example to an example selector as well.
    new_example = {"input": "big", "output": "small"}
    dynamic_prompt.example_selector.add_example(new_example)
    print(dynamic_prompt.format(adjective="enthusiastic"))

test3()

In [ ]:
from langchain_chroma import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # The VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # The number of examples to produce.
    k=1,
)

similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

def test1():
    # Input is a feeling, so should select the happy/sad example
    print(similar_prompt.format(adjective="worried"))


def test2():
    # Input is a measurement, so should select the tall/short example
    print(similar_prompt.format(adjective="large"))


def test3():
    # You can add new examples to the SemanticSimilarityExampleSelector as well
    similar_prompt.example_selector.add_example(
        {"input": "enthusiastic", "output": "apathetic"}
    )
    print(similar_prompt.format(adjective="passionate"))

test1()